In [ ]:
import vdblite
import numpy as np
from time import time
from uuid import uuid4
import sys
from pprint import pprint as pp
import dataset
import pickle

In [ ]:
file = open("simple_wikidata5m.pkl", "rb")
model = pickle.load(file)
entity2id = model.graph.entity2id
relation2id = model.graph.relation2id
entity_embeddings = model.solver.entity_embeddings
relation_embeddings = model.solver.relation_embeddings

In [ ]:
alias2entity = dataset.wikidata5m.load_alias("./entity.txt.gz")

In [ ]:
entity_db = vdblite.Vdb()
id2entity = model.graph.id2entity

In [ ]:
entity2alias = {v: k for k, v in alias2entity.items()}

In [ ]:
print(type(entity2alias))
for i in entity2alias:
    print(i, entity2alias[i])
    break

In [ ]:
for i in range(len(entity_embeddings)):
    entity_db.add({'entity_embedding': entity_embeddings[i], 'entity_id': i})

In [ ]:
entity_db.details()

In [ ]:
# Pass the output of query_to_kg_embeddings
# 
def get_neighbourhood(query_embeddings):
    neighbours = {}
    for qe in query_embeddings:
        nearest = entity_db.search(query_embeddings[qe], field='entity_embedding', count=3)
        print("Nearest: ", len(nearest))
        for n in nearest:
            alias = entity2alias[id2entity[n["entity_id"]]]
            neighbours[alias] = n["entity_embedding"]

    return neighbours

In [ ]:
temp = entity_embeddings[entity2id[alias2entity["steve jobs"]]]
query_embedding = {"steve jobs": temp}

In [ ]:
neighbours = get_neighbourhood(query_embeddings=query_embedding)
for n in neighbours:
    print(n)